# Python Worksheet: Maximum Likelihood for a Nonlinear Problem

Dear students,

I am sorry for handing out the worksheet so late. My team and I tried hard to get a useful exercise for MLE. MLE is a crucial tool for all data science. In addition, our standard problem set was handed out as part of the python lecture notes. We hence had to come up with something useful, yet, also doable at the BSc level.

The following exercise is the result. We will likely spend some time discussing your and my work during the upcoming prof cafe.


If a significant number of learning groups communicate back that the task is too hard to accomplish till monday, I am willing to extend the final submission by one week. Yet, there will be (a then shorter) PS on vol modeling ARCH due in the same week.


## Remember this Crucial FACT

**Do NOT numerically optimize a Gaussian likelihood for a LINEAR model. Instead, take the ML parameter estimates from an OLS fit AND the ML estimate for the volatility from the respective residual**

\begin{equation}
\sqrt{\frac{1}{T} \epsilon_{ols}' \epsilon_{ols}}
\end{equation}

##  A Useful Exercise had to be Non-Linear

Reason: see above

## G.0 Set-up of the Python Challenge

Have you ever wondered how central banks, banks, hedge funds and investors know whether a government bond is fairly priced, overpriced or undervalued? Well, they use the approach of "net present value" to figure out what the price of a bond should be. To do so, one has to get accurate forecasts of how the short rate and risk premia move into the future. A popular approach relies on so called "short rate models". These models fit the time series of the short rate to a realistic, yet convenient, parametrization. This allows to obtain a forecast of where short rates will be in the future. You need to do the same for the risk premium. You might wonder why do government bonds pay a risk premium. Well, the longer the maturity of the bond, the higher the interest rate sensitivity and hence, the higher (usually) the (maturity) premium. That premium compensates for the fact that future realized interest rates will be different from todays expected future rates. Based on the CAPM, you could say that interest rate risk is systematic and hence pays a risk premium. That is for the background.

A very popular short rate model is called "Vasicek model". That is because a mathematician, Dr. Vasicek, derived in the 1980s the term structure of interest rates under the assumption that the short rate follows a Gaussian distribution. It is fair to says that the assumed short rate follows a G-LRM. Based on stochastic calculus and the principle of net present value and no arbitrage, Dr. Vasicek derived the price of risk-free (non defaultable) bonds for different maturities. 

The Gaussian PDF of the short rate is

\begin{equation}
r_t | F_{t-1} \sim \mathcal N\left(\theta^P (1- e^{-\kappa^P})+e^{-\kappa^P} r_{t-1}, \sigma^2 * \frac{1-e^{-2\kappa^P}}{2\kappa^P}\right).
\end{equation}

Note, $\theta^P$ is the long-run mean of $r$, $kappa^P$ is the speed of mean reversion and $\sigma$ is the instantaneous standard deviation of the interest rate shock. As the model of Vasicek is written in continuous time, the above PDF takes several exponentials and ratios, which are the result of an integration that is NOT subject to our BSc course.

Also, Dr. Vasicek derived that the arbitrage-free price of a bond at time $t$ with maturity in $\tau$ periods, $P_t(\tau)$ coincides with

\begin{equation}
P_t(\tau) = e^{A(\tau) - B(\tau) \times r_t}
\end{equation}

with
\begin{align*}
B(\tau) &:= \frac{1-e^{-\kappa^Q \tau}}{\kappa^Q} \\
A(\tau) &:= (\theta^Q - \frac{\sigma^2}{2 (\kappa^Q)^2}) \times (  B(\tau) - \tau ) - \frac{\sigma^2}{4\kappa^Q} B^2(\tau) 
\end{align*}


Note, from the CAPM and NPV you know that prices of financial assets are discounted by the risk-free rate plus risk premium. The parameters $\theta^Q$ and $\kappa^Q$ differ from $\theta^P$ and $\kappa^P$, respectively. The former are adjusted by risk premiums. Intuitively, it is correct to think that the former are risk premium adjusted expressions of the latter. E.g. if the long-run mean of the short rate was at 2\%, it could be that bonds are priced as if the long-run rate was at 3\%. This means the price is lower than if the future payoff was discounted by 2\%. That extra reduction in price can be correctly interpreted as the result of a risk premium. That was to show you that the so called "Q parameters" are risk premium adjusted (and extracted from prices), while the $P$ parameters are extracted from empirical time-series data. I provide a formal and intuitive introduction into these asset pricing concepts in an upcoming MSc class on financial machine learning.

For now, you need only to keep in mind that the parameters of the model are $\kappa^P, \theta^P$ for the short rate PDF and $\theta^Q, \kappa^Q$ for the bond prices. The parameter $\sigma$ affects both, the short rate PDF and the bond price PDF.

Note, prices are non-stationary! Hence, we work with continuousy compounded yields
\begin{align*}
y_t(\tau) &:= - \frac{1}{\tau} \ln P_t(\tau)  
\end{align*}

In [15]:
import numpy as np
import pandas as pd
import scipy.optimize
import matplotlib.pyplot as plt

class OLS:

    def __init__(self,X,y):
        """
        Constructor
        :param X: np. matrix with X values
        :param y: np.matrix with y values
        """
        self.X = X
        self.y = y

        return None

    def runOLS(self):
        """
        Computation of regression
        :return:
        """
        # compute regression beta and residuals


        # calculate (x * x)^-1 with transosing and inverse functions
        X_X_invers = (self.X.getT() * self.X).getI()

        # calculate the beta ols estimate
        self.beta_ols = X_X_invers * self.X.getT() * self.y
        # print(self.beta_ols.shape)

        # calcuate the residuals
        self.resid = y - X * self.beta_ols


        # calculate the volatility of the residual

        # calculate the variance
        var_res = 1 / (len(y) - len(self.beta_ols)) * self.resid.getT() * self.resid

        # calcualte the standard error
        self.vol_res = np.sqrt(var_res)


        # standard errors and t-stat of beta estimates

        # covariance matrix ov betas
        var_b = var_res[0,0] * X_X_invers

        # sd of betas laying on the diagonal of the covariance matrix
        self.vol_b = np.sqrt(var_b.diagonal())

        # TODO: Matrix checking

        # check the dimensions
        # beta as a 2 x 1 row vector
        # print(self.beta_ols.shape)
        # st must therefore also be a row vector for element wise division
        # print(self.vol_b.shape)

        # reshape
        self.vol_b = self.vol_b.reshape(-1,1)

        # calcualte the t stat
        self.t_stat =  self.beta_ols / self.vol_b


        # adjusted r^2

        # calcuate the sample variance
        help = y - y.mean()
        var_y = (help.T * help) / (len(y) - 1)

        # calcuate the adjusted r squared
        self.adjust_r_squared = 1 - (var_res[0,0]/var_y[0,0])

        return None

    def summaryStats(self):

        # runOls
        # self.runOLS()

        # display results
        results_df = pd.DataFrame(columns=["Betas", "vol Betas","t-stat"])

        help_list1 = self.beta_ols.tolist()
        help_list2 = self.vol_b.tolist()
        help_list3 = self.t_stat.tolist()

        help_list1 = [round(i[0],4) for i in help_list1]
        help_list2 = [round(j[0],4) for j in help_list2]
        help_list3 = [round(k[0],4) for k in help_list3]

        results_df["Betas"] = help_list1
        results_df["vol Betas"] = help_list2
        results_df["t-stat"] = help_list3

        print("-" * 10)
        print("Summary Statistics:")
        print(results_df)
        print("-" * 10)
        print(f"The residuals volatility is: {round(self.vol_res[0,0],4)}")
        print(f"The adjusted R^2 is: {round(self.adjust_r_squared,4)}")
        print("-" * 10)

        return None


def set_time_index(df, timecolname):

    """This function sets the time col as index and makes sure it's a datetime object.

    :param df: full Dataframe
    :param timecolname: colname of the column that has time information in it
    :return: full Dataframe
    """
    # take the time column and convert it to a datetime object
    df[timecolname] = pd.to_datetime(df[timecolname])

    # set the index of the DF as the time Column
    df.set_index(timecolname, inplace = True)

    return df

## G.1 Data

You will work with the same interest rate data than a couple of weeks ago. The rates are in "GovBondYields.xls". 

Take the 12-, 60 and 120 months interest rates as your yields of interest, $y$. Take the 3-month yield as the short rate $r$.

**Units:** Divide the rates data by 1200 to arrive at monthly values in decimals. Work with these units.

In [16]:
# import the needed data
df_bond = pd.read_excel("GovBondYields.xls",sheet_name="Rates",header=0)
df_bond = set_time_index(df_bond,"Date")

# print(df_bond.head())
# print(df_bond.tail())
# print(df_bond.describe())
# plt.plot(df_bond)
# plt.show()

# Divide the rates by 1200 to arrive at monthly values
df_bond = df_bond / 1200
# print(df_bond.head())

# get the names of the columns
colnames = df_bond.columns.tolist()

# get the wanted columns as pandas Series
month_3_interst_r = df_bond[colnames[0]]
month_12_interst = df_bond[colnames[2]]
month_60_interst = df_bond[colnames[5]]
month_120_interst = df_bond[colnames[7]]
# print(month_3_interst_r.head())

# get some intuition for r : monthly interest rate
# print(month_3_interst_r.describe())
# plt.plot(month_3_interst_r)
# plt.show()

# get the time horizon
T = df_bond.shape[0]
# print(T)

## G.2 Financial Data Science Challenge

Your data science task is to extract the time-series of the expected market price of risk that is priced into bonds. You can consider the latter to be the priced-in Expected Sharpe Ratio in bonds (expected risk premium per unit of risk). The precise formula is given below. 


## G.3 Battle Plan

The battle plan is as follows. 

1. Write down the joint log likelihood function for $r_t$ and $y_t(\tau), \tau \in [12,60,120]$. For that assume a pairwise independent Gaussian measurement error for all yields. Further assume each of these measurement errors shares the same volatility, which we call
\begin{equation}
\sigma_y.
\end{equation}
Note, $\sigma_y$ will be part of the parameter vector over which you optimize.

Note, you can interpret observed yields as Vasicek (model)-implied yield $y_t(\tau)$ plus a Gaussian measurement error that is drawn from $N(0,\sigma^2_y)$.

Note: the above assumption implies that the joint log likelihood function can be treated as the sum of 4 pairwise independent log likelihood functions. The independence refers to the respective residual being independent of the residuals of the other log likelihood. The parameters do of course affect all of the log likelihoods; although strictly speaking, $\kappa^P, \theta^P$ affect only the log likelihood for $r$, while $\kappa^Q, \theta^Q$ affect only the log likelhood for the 3 yields.

2. Write a python function that implements the joint log likelihood function. More recent research has shown you could indeed fit the PDF for $r$ independently for the PDF of $y$. Yet, here, we optimize the joint log likelihood.

3. Play a bit around with starting values for the optimization. That simple example reveals already, that you need to choose starting values carefully to end up in the global optimum. Regardless of your playing around, use at least the following procedure to find smart starting values

3.1 fit $r$ to an OLS and recover the ML estimates for $\kappa^P, \theta^P, \sigma$. Make these parameters be the respective starting values $\kappa^P, \theta^P, \sigma$ in  the joint log likelihood optimization. 

3.2 for the starting values of the Q parameters, we assume a 0 risk premium. Hence $\kappa^Q_{startValue} = \kappa^P_{startValue}$ and $\theta^Q_{startValue} = \theta^P_{startValue}$. Double check: inside the optimization, Q and P parameters can differ(!). P parameters affect $r$ while $Q$ parameters affect yields (!)

3.3 For the starting value of $\sigma_y$ we do the following: Regress (OLS) the 60month yield onto $r$ and take the volatility of the residual as the starting value for $\sigma_y$. 


4. The optimization routine is similar to the ipynb from class

4.1 use sco.minimize

4.2 use L-BFGS-B as the optimization routine

4.3 use a tol of 1e-8

4.5 minimize the negative joint log likelihood function

4.6 the upper bound for all parameters is 100. 

4.7 the lower bound for all parameters is -100, except for $\sigma$ and $\sigma_y$ who lower bound we set to 0.000001.

5. Estimate the model parameters of Vasicek.

6. Inside the vasicek model, the priced-in market price of risk (Sharpe ratio) in bonds is

\begin{equation}
RP_t = \lambda_0 + \lambda_1 \times r_t
\end{equation}

with
\begin{align*}
\lambda_0 &:= \frac{\kappa^P \theta^P - \kappa^Q \theta^Q}{\sigma_r} \\
\lambda_1 &:= \frac{\kappa^Q - \kappa^P}{\sigma_r}.
\end{align*}

Here, you see that spread of Q and P parameters captures risk premium information. A formal explanation of that is not doable in the BSc course on Financial Data Science.

In [17]:
# SEE: https://en.wikipedia.org/wiki/Vasicek_model

"Note simga stand vor volatility"

# implement the model
def vasicek_model(rt,tau,k_q,o_q,sigma_r):

    """ This function implements the calcualation of the implied yield
    :param rt: pandas Series of the short rate rt
    :param tau:  maturity in tau periods
    :param k_q: param of model
    :param o_q: param of model
    :param sigma_r: parm of model
    :return: model implied yield in a 1 x T numpy matrix
    """

    # convert rt to a numpy matrix : Loose the dates but usefull for further calculation
    rt = np.matrix(rt)

    # check the dimesnions : 1 x T : row vector
    # print(rt.shape)

    b_r = (1 - np.exp(-k_q*tau)) / k_q

    a_r = (o_q -(sigma_r)/(2* k_q**2)) * (b_r - tau - sigma_r / (4*k_q) * b_r**2)

    # price of a bond at time t with maturity tau
    p_t = np.exp(a_r - b_r * rt)

    # continuoisly compounded yields
    y_t = (- 1/tau) * np.log(p_t)

    # check the dimesnions : 1 x T : row vector
    # print(y_t.shape)

    return y_t

# a = vasicek_model(rt = month_3_interst_r,tau = 120,k_q = 1,o_q = 1,sigma_r = 0.0001)
# print(a)

In [18]:
# we need the joint log likelyhood function of rt
# Assumption: iid Gaussion white noise errors with et - N(0,sig_e^2)

def joint_log_likelyhood_rt(rt,op,kappa,sigma_r):

    """THis function computes the jll of rt with a gaussian pdf
    :param rt: pandas Series of the short rate rt
    :param op: param of model, long run mean
    :param kappa: param of model, speed of the reversion
    :param sigma_r: param of model, instantaneous standard deviation of the interest rate shock
    :return: joint log likelyhood of all r_t = (r_1,...,rT) : skalar
    """

    # as is creates the same results

    # shift the ts one day back
    # rt_1 = rt.shift(1).dropna()
    rt_1 = rt[0:T-1]

    # convert rt to a numpy matrix
    rt_1 = np.matrix(rt_1)

    # remove the first datapoint from rt
    rt = rt[1:T]

    # convert rt to a numpy matrix
    rt = np.matrix(rt)

    # check the dimesnions : 1 x T : row vector
    # print(rt.shape)

    # calcuate the mu
    # mu = op * (1-np.exp(-kappa)) + np.exp(-kappa) * rt[0]
    # print(rt-mu)
    mu = op * (1-np.exp(-kappa)) + np.exp(-kappa) * rt_1

    # check the dimesnions : 1 x T : row vector
    # print((rt-mu).shape)

    # calcualte the sigma
    sigma = sigma_r * ((1-np.exp(-2 * kappa))) / (2 * kappa)
    # print(sigma)

    # in matrix notation
    log_joint_likely = - T/2 * np.log(2 * np.pi) - T/2 * np.log(sigma) - 1 / (2 * sigma) * (rt-mu) * (rt-mu).getT()

    return float(log_joint_likely)

a = joint_log_likelyhood_rt(rt = month_3_interst_r,op = 1,kappa = 1,sigma_r = 0.001)
print(a)

-284018.6574564117
-284018.6574564117


In [19]:
# we need the joint log likelyhood function of yt

def joint_log_likelyhood_yt(yt,sigma_y,rt,tau,k_q,o_q,sigma_r):

    """THis function computes the jll of rt with a gaussian pdf
    :param yt: pandas Series of the yield
    :param sigma_y: instantaneous standard deviation of the interest rate shock
    :param rt: pandas Series of the short rate rt
    :param tau:  maturity in tau periods
    :param k_q: param of model
    :param o_q: param of model
    :param sigma_r: parm of model
    :return: joint log likelyhood of all y_t = (y_1,...,yT) : skalar
    """

    # convert yt to a numpy matrix
    yt = np.matrix(yt)

    # check the dimesnions : 1 x T : row vector
    # print(yt.shape)

    # calcuate the mu
    # you can interpret observed yields as Vasicek (model)-implied yield plus a Gaussian measurement error
    mu = vasicek_model(rt,tau,k_q,o_q,sigma_r)
    # print(mu)

    # check the dimesnions : 1 x T : row vector
    # print(mu.shape)

    # check the dimesnions : 1 x T : row vector
    # print((yt-mu).shape)

    # calcualte the sigma
    sigma = sigma_y

    # in matrix notation
    log_joint_likely = - T/2 * np.log(2 * np.pi) - T/2 * np.log(sigma) - 1 / (2 * sigma) * (yt -mu) * (yt-mu).getT()

    return float(log_joint_likely)

# a = joint_log_likelyhood_yt(yt = month_120_interst,sigma_y = 0.0001,rt = month_3_interst_r,tau = 120,k_q = 1,o_q = 1,sigma_r = 0.0001)
# print(a)

In [20]:
def joint_likelyhood_yt_rt(param_vec):

    """This function computes the joint likelyhood function
    :param op: parm of model
    :param kappa: parm of model
    :param o_q: parm of model
    :param k_q: parm of model
    :param sigma_y: parm of model
    :param sigma_r: parm of model
    :return: negative log likelyhood with given unknown parameters: skalar
    """

    # order of the parm vector
    # [op,kappa,o_q,k_q,sigma_y,sigma_r]

    # get the parameters
    op = param_vec[0]
    kappa = param_vec[1]
    o_q = param_vec[2]
    k_q = param_vec[3]
    sigma_y = param_vec[4]
    sigma_r = param_vec[5]

    # IDEA: joint log likelihood function can be treated as the sum of 4 pairwise independent log likelihood functions

    # function for the short rate
    L_r = joint_log_likelyhood_rt(rt =month_3_interst_r,op = op,kappa = kappa,sigma_r = sigma_r)
    print(L_r)
    L_y_12 = joint_log_likelyhood_yt(yt = month_12_interst,sigma_y =sigma_y ,rt = month_3_interst_r,tau = 12,k_q = k_q,o_q = o_q,sigma_r = sigma_r)
    print(L_y_12)
    L_y_60 = joint_log_likelyhood_yt(yt =month_60_interst,sigma_y =sigma_y ,rt = month_3_interst_r,tau = 60,k_q = k_q,o_q = o_q,sigma_r = sigma_r)
    print(L_y_60)
    L_y_120 = joint_log_likelyhood_yt(yt =month_120_interst,sigma_y =sigma_y ,rt = month_3_interst_r,tau = 120,k_q = k_q,o_q = o_q,sigma_r = sigma_r)
    print(L_y_120)
    # compute the total joint likelyhood function
    L = L_r + L_y_12 + L_y_60 + L_y_120

    # return the negative log likelyhood
    return L * -1

In [21]:
# parameters as default: are getting overwritten
op = 1
kappa = 1
o_q = 1
k_q = 1
sigma_y = 0.001
sigma_r = 0.001

# all in one vector
param_vec = [op,kappa,o_q,k_q,sigma_y,sigma_r]

L = joint_likelyhood_yt_rt(param_vec)
print(L)

# get the length: number of unknown parameters
K = len(param_vec)

-284018.6574564117
-258211.46627526206
-297070.9561077101
-302054.62736152345
1141355.7072009072
-284018.6574564117
-258211.46627526206
-297070.9561077101
-302054.62736152345
1141355.7072009072


In [22]:
# find the respective starting parameters op, kappa, sigma_r: we need to retrieve 3 values

# op is the long-run mean of r, All future trajectories of r will evolve around a mean level b in the long run: intercept
# kappa is the speed of mean reversion, characterizes the velocity at which such trajectories will regroup around {\displaystyle b}b in time : slope
# sigma_r is the instantaneous standard deviation of the interest rate shock : variance of residuals

# make use of the AR(1) structure

df_for_regression_r = pd.DataFrame(columns = ["month_3"])
df_for_regression_r["month_3"] = df_bond[colnames[0]]

# get the y column: exclude the first x values
y = df_for_regression_r["month_3"][1:T]

# convert to numpy matrix
y = np.matrix(y)

# need to transpose the vector
y = y.reshape(T-1,1)

# create X : Lag 1
lag1 = df_for_regression_r["month_3"][0:T-1]

# create X : Intercept
ones = np.ones(T-1)

# combine to a dataframe
x_df = pd.DataFrame(ones)
x_df["Lag1"] = lag1.values

# convert to a numpy matrix
X = np.matrix(x_df)

# create the model
model = OLS(X,y)

# fit the model
model.runOLS()

# get the results
model.summaryStats()

"Care adjust the values: See the PDF Formula"
# y = a+ bx + e
# with a = o_p * (1 -ek_p )
# with b = e -k_p
# with e =  sig2 * 1-....

# intercept
op = float(model.beta_ols[0])
print(op)


# slope
kappa = float(model.beta_ols[1])
print(kappa)


# resid
sigma_r  = float(model.vol_res)
print(sigma_r)


----------
Summary Statistics:
    Betas  vol Betas    t-stat
0  0.0001     0.0000    2.1704
1  0.9852     0.0064  152.9405
----------
The residuals volatility is: 0.0004
The adjusted R^2 is: 0.9741
----------
6.890460011518503e-05
0.9852150542451509
0.00037794053136844924
----------
Summary Statistics:
    Betas  vol Betas    t-stat
0  0.0001     0.0000    2.1704
1  0.9852     0.0064  152.9405
----------
The residuals volatility is: 0.0004
The adjusted R^2 is: 0.9741
----------
6.890460011518503e-05
0.9852150542451509
0.00037794053136844924


In [23]:
# find the respective starting parameters o_q,k_q

# starting with zero risk premium, thus:
o_q = op
k_q = kappa

In [24]:
# find the respective starting parameters sigma_y

df_for_regression_y = pd.DataFrame(columns = ["month_3","month_120"])
df_for_regression_y["month_3"] = df_bond[colnames[0]]
df_for_regression_y["month_60"] = df_bond[colnames[5]]

y = df_for_regression_y["month_3"]

# convert to numpy matrix
y = np.matrix(y)

# need to transpose the vector
y = y.reshape(T,1)

# create X : Intercept
ones = np.ones(T)

# combine to a dataframe
x_df = pd.DataFrame(ones)
x_df["month_60"] = df_for_regression_y["month_60"].values

# convert to a numpy matrix
X = np.matrix(x_df)

# create the model
model = OLS(X,y)

# fit the model
model.runOLS()

# get the results
model.summaryStats()

# resid
sigma_y  = float(model.vol_res)
print(sigma_y)

----------
Summary Statistics:
    Betas  vol Betas   t-stat
0 -0.0008     0.0001  -9.9178
1  0.9638     0.0139  69.3643
----------
The residuals volatility is: 0.0008
The adjusted R^2 is: 0.8852
----------
0.0007958932001805073
----------
Summary Statistics:
    Betas  vol Betas   t-stat
0 -0.0008     0.0001  -9.9178
1  0.9638     0.0139  69.3643
----------
The residuals volatility is: 0.0008
The adjusted R^2 is: 0.8852
----------
0.0007958932001805073


In [25]:
# now start with the numerical optimazation of the negative joint likelihood function

# starting values
starting_v = [op,kappa,o_q,k_q,sigma_y,sigma_r]

# lower bounds
lb = -100 * np.ones(K)
lb[K-2] = 0.000001
lb[K-1] = 0.000001

# upper bounds
ub = 100 * np.ones(K)

# bounds:
bounds = tuple((lb[x],ub[x]) for x in range(0,K))
print(bounds)

((-100.0, 100.0), (-100.0, 100.0), (-100.0, 100.0), (-100.0, 100.0), (1e-06, 100.0), (1e-06, 100.0))
((-100.0, 100.0), (-100.0, 100.0), (-100.0, 100.0), (-100.0, 100.0), (1e-06, 100.0), (1e-06, 100.0))


In [26]:
# optimazation

# returns the optimized values
mle_estimate = scipy.optimize.minimize(fun = joint_likelyhood_yt_rt,x0 = starting_v,bounds = bounds, method = "L-BFGS-B",tol = 1e-8)

print(mle_estimate)

# get the results as an array
optimize_res = mle_estimate.x
# print(optimize_res)
# print(joint_likelyhood_yt_rt(optimize_res) * -1)

2129.558003928598
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558067334117
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558005757024
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558003928598
1645.4665892719927
1642.345912505934
1641.5349533662168
2129.558003928598
1645.4665564861677
1642.3458710354116
1641.5349099396985
2129.558003928598
1645.4627584046423
1642.3421121464285
1641.5311612372282
2129.5502042263142
1645.4665396039766
1642.3458496541555
1641.5348875468214
-623945870511542.0
-inf
-inf
-inf
-623945870386747.5
-inf
-inf
-inf


<ipython-input-3-aa95eb067276>:31: RuntimeWarning: divide by zero encountered in log
  y_t = (- 1/tau) * np.log(p_t)
c:\users\win10\pycharmprojects\elasticstack\venv\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


-623945870449147.5
-inf
-inf
-inf
-623945870511542.0
-inf
-inf
-inf
-623945870511542.0
-inf
-inf
-inf
-623945870511542.0
-inf
-inf
-inf
-617768188625238.9
-inf
-inf
-inf
2129.558003928598
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558067334117
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558005757024
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558003928598
1645.4665892719927
1642.345912505934
1641.5349533662168
2129.558003928598
1645.4665564861677
1642.3458710354116
1641.5349099396985
2129.558003928598
1645.4627584046423
1642.3421121464285
1641.5311612372282
2129.5502042263142
1645.4665396039766
1642.3458496541555
1641.5348875468214
      fun: -7058.905341366528
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.34055177e+03, -1.82842631e+02, -1.17706213e+04, -2.33485479e+00,
        1.13056496e+06,  7.86033526e+05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 21
      nit: 1
   

<ipython-input-17-aa95eb067276>:31: RuntimeWarning: divide by zero encountered in log
  y_t = (- 1/tau) * np.log(p_t)
c:\users\win10\pycharmprojects\elasticstack\venv\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


-617768188625238.9
-inf
-inf
-inf
2129.558003928598
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558067334117
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558005757024
1645.4665564904894
1642.3458710229163
1641.5349099245236
2129.558003928598
1645.4665892719927
1642.345912505934
1641.5349533662168
2129.558003928598
1645.4665564861677
1642.3458710354116
1641.5349099396985
2129.558003928598
1645.4627584046423
1642.3421121464285
1641.5311612372282
2129.5502042263142
1645.4665396039766
1642.3458496541555
1641.5348875468214
      fun: -7058.905341366528
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.34055177e+03, -1.82842631e+02, -1.17706213e+04, -2.33485479e+00,
        1.13056496e+06,  7.86033526e+05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 21
      nit: 1
     njev: 3
   status: 0
  success: True
        x: array([6.89046001e-05, 9.85215054e-01, 6.89046001e-05, 9.85215054e-01,
       7.958932

In [27]:
def get_priced_in_risk(rt,optimize_res):

    """This function calculates the  priced-in market price of risk
    :param rt: pandas Series of pandas Series of the short rate rt
    :param optimize_res: array with optimized values
    :return:
    """

    # convert rt to a numpy matrix
    # rt = np.matrix(rt)

    # check the dimesnions : 1 x T : row vector
    # print(rt.shape)

    # get the estimated values
    op = optimize_res[0]
    kappa = optimize_res[1]
    o_q = optimize_res[2]
    k_q = optimize_res[3]
    sigma_r = optimize_res[K-1]

    # needed help formulas : skalars

    # is positive and large
    lam0 = (kappa * op - k_q * o_q) / sigma_r
    print(lam0)

    # is negative and big
    lam1 = (k_q - kappa) / sigma_r
    print(lam1)

    # sharpe ratio
    rtp = lam0 + lam1 * rt

    return rtp

In [28]:
priced_in_risk = get_priced_in_risk(month_3_interst_r,optimize_res)
print(priced_in_risk)

0.0
0.0
Date
1954-04-01    0.0
1954-05-01    0.0
1954-06-01    0.0
1954-07-01    0.0
1954-08-01    0.0
             ... 
2005-12-01    0.0
2006-01-01    0.0
2006-02-01    0.0
2006-03-01    0.0
2006-04-01    0.0
Name: 3, Length: 625, dtype: float64
0.0
0.0
Date
1954-04-01    0.0
1954-05-01    0.0
1954-06-01    0.0
1954-07-01    0.0
1954-08-01    0.0
             ... 
2005-12-01    0.0
2006-01-01    0.0
2006-02-01    0.0
2006-03-01    0.0
2006-04-01    0.0
Name: 3, Length: 625, dtype: float64
